In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from collections import defaultdict
import pickle
import torch
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
import torchvision

In [3]:
with open("/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/ReLUs/ReLUs_train.pkl", 'rb') as iffile:
    features = pickle.load(iffile)

In [4]:
layer = 'flatten'
features = features[layer]
features_matrix = None

for i in range(len(features)):
    if isinstance(features_matrix, torch.Tensor):
        features_matrix = torch.cat((features_matrix, features[i]))
    else:
        features_matrix = features[i]

features = None

In [5]:
features_matrix = torch.flatten(features_matrix, start_dim=1, end_dim=-1)

In [6]:
# Load adversarial samples
with open("/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/ReLUs/ReLUs_test_adversarial.pkl", 'rb') as iffile:
    adv_features = pickle.load(iffile)

layer = 'flatten'
adv_features = adv_features[layer]
adv_features_matrix = None

for i in range(len(adv_features)):
    if isinstance(adv_features_matrix, torch.Tensor):
        adv_features_matrix = torch.cat((adv_features_matrix, adv_features[i]))
    else:
        adv_features_matrix = adv_features[i]

adv_features = None

In [18]:
X_ben = features_matrix[0:512, :]
y_ben = np.ones((512))

#X_test = features_matrix[:15000, :]
X_adv = adv_features_matrix
y_adv= -np.ones((512))

In [17]:
X_train = np.concatenate((X_ben, X_adv), axis=0)

In [19]:
y_train = np.concatenate((y_ben, y_adv), axis=0)

In [20]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [21]:
X_train, y_train = unison_shuffled_copies(X_train, y_train)

In [93]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC

clf = SVC(C=0.7, gamma=0.075)
clf.fit(X_train, y_train)

SVC(C=0.7, gamma=0.075)

In [94]:
res_adv = clf.predict(X_adv)

In [95]:
len(np.where(res_adv == -1)[0]) / 512

0.90625

In [96]:
res_ben = clf.predict(X_ben)

In [97]:
len(np.where(res_ben == 1)[0]) / 512

0.927734375

In [1]:
import pickle

In [2]:
with open("/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_d2a45a4dd02a5e037e5954b82387e666/RBF/RBF_CW_5120.pkl", 'rb') as iff:
    clf = pickle.load(iff)

In [3]:
clf

SVC(C=0.7, gamma=0.075)

In [27]:
# Quantize
# Comput per-dimension mean

In [ ]:
dim_mean = features_matrix.mean(dim=0)

In [ ]:
binarized_matrix = torch.where(features_matrix >= dim_mean, torch.ones_like(features_matrix), torch.zeros_like(features_matrix))

In [ ]:
features_matrix = None

In [ ]:

torch.cuda.empty_cache()

In [ ]:
binarized_matrix = binarized_matrix.cpu().numpy()

In [ ]:


model = OneClassSVM(kernel='rbf', gamma=0.0000000001)
model.fit(features_matrix)

In [16]:
# Hook stuff
import pickle

In [17]:
hook_path = "/home/zsarwar/Projects/SparseDNNs/hooks.pickle"
with open(hook_path, 'rb') as ho_file:
    hook_features = pickle.load(ho_file)

In [8]:
from collections import defaultdict
import pickle
import torch
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
import numpy as np
import pandas as pd

from sklearn.svm import OneClassSVM
import torchvision
from torchvision.models import resnet18
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC
import os

In [9]:
def load_data(path, label):
    with open(path, 'rb') as iffile:
        features = pickle.load(iffile)
    layer = 'flatten'
    features = features[layer]
    features_matrix = None

    for i in range(len(features)):
        if isinstance(features_matrix, torch.Tensor):
            features_matrix = torch.cat((   features_matrix, features[i]))
        else:
            features_matrix = features[i]

    features = None    
    features_matrix = torch.flatten(features_matrix, start_dim=1, end_dim=-1)
    y = np.empty(features_matrix.shape[0])
    y.fill(label)
    return [features_matrix, y]


In [10]:
def unison_shuffled_copies(x_ben, y_ben, x_adv, y_adv):
    X = np.concatenate((x_ben, x_adv), axis=0)
    y = np.concatenate((y_ben, y_adv), axis=0)
    assert len(X) == len(y)
    p = np.random.permutation(len(X))
    return X[p], y[p]

In [13]:
train_adversarial = "/home/zsarwar/Projects/SparseDNNs/features.pickle"

In [14]:
train_adversarial = load_data(train_adversarial, 1)
